In [ ]:
https://www.verizonwireless.com/smartphones/samsung-galaxy-s9/

In [67]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pathlib import Path
from bs4 import BeautifulSoup

driver = webdriver.Chrome()
driver.get("https://www.verizonwireless.com/smartphones/samsung-galaxy-s9/")

# need to navigate through reviews.
# problem for this website is that the url doesn't change no matter what I navigate through, beacuse it is a dynamic website
# but selenium can do it


#this is to get to the reviews from the original url
review_button_path = '//*[@id="reviewsLink"]'
wait = WebDriverWait(driver, 5)
review_button = wait.until(EC.visibility_of_element_located((By.XPATH, review_button_path)))
driver.execute_script("arguments[0].click();", review_button)

In [68]:
# this below retreives all the text in the review container
# but don't need it....

# get the reviews and rating for the first page
reviews = []
ratings = []


review_list = driver.find_elements_by_css_selector('.NHaasDS55Rg.fontSize_12.padBottom15.wrapText')
for el in review_list:
    reviews.append(el.find_element_by_tag_name('span').text)

    
rating_list = driver.find_elements_by_css_selector('.displayInlineBlock.positionRelative.fontSize_14')
for st in rating_list:
    ratings.append(st.find_element_by_class_name('a11y-hidden').text)

In [69]:
index= len(ratings)
ratings[7:index-1]


['3 out of',
 '4 out of',
 '4 out of',
 '3 out of',
 '2 out of',
 '4 out of',
 '1 out of',
 '4 out of',
 '3 out of']

In [70]:
from selenium.common import exceptions
import time

# get the reviews starting from the 2nd page
# until there is no button to click
# there are about 37ish pages total since there are about 1100 reviews
# (probably) 30 pages per reviews except for the first page
i=2
reviews_comb=[]
ratings_comb=[]

while(True):
    try :
        next_review_path = '//*[@id="reviews"]/div/div[3]/div/ul/li[6]/a/span[1]' 
        review_next_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, next_review_path)))
        driver.execute_script("arguments[0].click();", review_next_button)
        time.sleep(3)
  
        # append the reviews and ratings to a list
        review_list = driver.find_elements_by_css_selector('.NHaasDS55Rg.fontSize_12.padBottom15.wrapText')
        for el in review_list:
            reviews_comb.append(el.find_element_by_tag_name('span').text)

    
        rating_list = driver.find_elements_by_css_selector('.displayInlineBlock.positionRelative.fontSize_14')
        for st in rating_list:
            ratings_comb.append(st.find_element_by_class_name('a11y-hidden').text)
        print(i)
        i=i+1
    
    #except exceptions.StaleElementReferenceException:
    except exceptions.StaleElementReferenceException as e:
        raise e
    
    except exceptions.TimeoutException :
        break
    

2
3
4
5
6
7
8


In [73]:
merged_reviews = reviews + reviews_comb
merged_ratings = ratings + ratings_comb

In [76]:
import pandas as pd

df_rating = pd.DataFrame({"review" : merged_reviews})
df_stars = pd.DataFrame({"rating" : merged_ratings})

galaxy_s9 = pd.concat([df_rating, df_stars], axis=1)

galaxy_s9['phone'] = "galaxy"
galaxy_s9['source'] = "vw.com"
galaxy_s9

cols = ['source','phone','review','rating']
galaxy_s9 = galaxy_s9.reindex(columns=cols)

galaxy_s9.to_csv('vw_galaxy.csv', index=False)